In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder \
         .master("spark://192.168.1.71:7077") \
         .appName("Project") \
         .config("spark.some.config.option", "some-value") \
         .getOrCreate()
        
sc = spark.sparkContext

In [35]:
#reading the csv file
df = spark.read.csv('Crimes.csv', header=True, inferSchema=True)

In [36]:
#df2 is 2.9 gb
df2 = df.union(df)


In [37]:
#df3 is 5.8 gb
df3 = df2.union(df2)

In [47]:
df4 = df3.union(df3)
df4.count()

52807112

In [39]:
#For printing the rdd just paas the rdd object to print_rdd function
def print_rdd(output):
    for (word,count) in output.collect():
        print("%s: %i" % (word,count))
    

In [40]:
def map_reduce_crimetype(df):
    crime_type = df.select('Primary Type')
    rddcrime = crime_type.rdd
    wordcounts= rddcrime.map(lambda w:(w,1)) \
        .reduceByKey(lambda a,b:a+b) 
    return wordcounts

In [41]:
rdd = map_reduce_crimetype(df4)
print_rdd(rdd)

Row(Primary Type='BATTERY'): 9637608
Row(Primary Type='MOTOR VEHICLE THEFT'): 2472208
Row(Primary Type='ARSON'): 87464
Row(Primary Type='ROBBERY'): 2003992
Row(Primary Type='CRIM SEXUAL ASSAULT'): 208800
Row(Primary Type='RITUALISM'): 184
Row(Primary Type='NON - CRIMINAL'): 304
Row(Primary Type='HUMAN TRAFFICKING'): 320
Row(Primary Type='STALKING'): 26064
Row(Primary Type='PUBLIC INDECENCY'): 1232
Row(Primary Type='HOMICIDE'): 72616
Row(Primary Type='THEFT'): 11055400
Row(Primary Type='DECEPTIVE PRACTICE'): 2015192
Row(Primary Type='OBSCENITY'): 4264
Row(Primary Type='NON-CRIMINAL'): 1160
Row(Primary Type='CONCEALED CARRY LICENSE VIOLATION'): 1496
Row(Primary Type='NON-CRIMINAL (SUBJECT SPECIFIED)'): 56
Row(Primary Type='INTERFERENCE WITH PUBLIC OFFICER'): 115464
Row(Primary Type='LIQUOR LAW VIOLATION'): 111200
Row(Primary Type='BURGLARY'): 3051888
Row(Primary Type='PROSTITUTION'): 543360
Row(Primary Type='WEAPONS VIOLATION'): 540824
Row(Primary Type='GAMBLING'): 114032
Row(Primary Typ

In [46]:
print(rdd.getNumPartitions())
print(rdd.count())

96
35


In [11]:
schemaPeople = spark.createDataFrame(wordcounts)

#renaming the column in dataframe
schemaPeople = schemaPeople.withColumnRenamed('_1','Primary Type')
schemaPeople = schemaPeople.withColumnRenamed('_2','Frequency')

schemaPanda = schemaPeople.toPandas()
display(schemaPanda)
schemaPanda.plot.bar(x ='Primary Type', y ='Frequency', colormap = 'summer')

,Primary Type,Frequency
0,"(INTIMIDATION,)",3832
1,"(NON-CRIMINAL,)",145
2,"(PUBLIC PEACE VIOLATION,)",47054
3,"(BATTERY,)",1204701
4,"(SEX OFFENSE,)",24392
5,"(NON-CRIMINAL (SUBJECT SPECIFIED),)",7
6,"(CONCEALED CARRY LICENSE VIOLATION,)",187
7,"(STALKING,)",3258
8,"(MOTOR VEHICLE THEFT,)",309026
9,"(OFFENSE INVOLVING CHILDREN,)",44061


In [33]:
spark.stop()